# Import libs und Daten

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
# XML-Lib laden
import xml.etree.ElementTree as ET

In [2]:
#url_xml = "./analyse_alt/rohdaten/wahl_gemeinden.xml"
url_xml = "./test_daten/F.210906.124700.GE.xml"
#url_xml = "./test_daten/F.210906.132800.WK.xml"
root = ET.parse(url_xml).getroot()

wahlergebnisse_2021 = pd.read_csv("./data/BTW_2017_GE.csv")

# Herauslesen Daten Dataframe long

In [3]:
#HELPER
# Ordnet die Beschreibung des Tags <Gruppenergebnis> einer Klarbzeichung zu
# diese Beschreibung ist unter dem Attribut Position abgespeichert

# Check ob sich in einem String eine Nummer befindet
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


In [11]:
# Dict Parteien 

In [4]:
# Offizielle Listennummer mit Parteiname BTW 2021
parteien_dict = {"1":"AfD", "2":"CDU", "3":"LINKE", "4":"SPD", "5":"FDP", "6":"GRÜNE", \
                # kleinere Parteien (sonstige)
                "7":"Tierschutzpartei", "8":"Die PARTEI", "9":"NPD", "10":"FREIE WÄHLER", \
                 "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
                 "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
                 "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
                 # kleinere Parteien ohne Landesliste
                 "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
                 "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
                 "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
                 "33": "Internationalistisches Bündnis", "34": "Internationalistisches Bündnis", \
                 "35": "Internationalistisches Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
                 "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"REICHEL",\
                 "42":"Röhder", "43":"UNO - Menschrecht - FRIEDEN für Alle", "44":"WIR sind LEIPZIGER", \
                 "45":"www.zeitkaufhaus.de Martin Bayer", \
                 # Bezeichner für gültige und ungültige Stimmen
                "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

                }

In [5]:
# Listennummer Parteien LTW 2019
lt_parties_dict = {
    "1":"CDU", "2":"LINKE", "3":"SPD", "4":"AfD", "5": "GRÜNE", "6":"NPD", "7":"FDP", \
    "8":"FREIE WÄHLER", "9":"Tierschutzpartei", "10":"PIRATEN", "11":"DIE PARTEI", "12":"BüSo", \
    "13":"ADPM", "14":"Blaue", "15":"KPD", "16":"ÖDP", "17":"Humanisten", "18":"PDV", \
    "19":"Gesundheitsforschung", "20":"andere1", "21":"andere2", \
    # Bezeichner für gültige und ungültige Stimmen
    "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"
    
}

In [6]:


# jedes elem in root ist der Tag <Gebietsergebnis>, in dem immer eine komplette Gemeinde steckt. 
def read_xml(root):
    
    # Variablen in Datensatz
    erststimmen = "DIREKT"
    zweitstimmen = "LISTE"


    
    # zwei Dataframes werden erstellt, eins für die Ergebnisse der Parteien, 
    # ein anderes für Wahlstatistiken etc. 
    
    # DF Ergebnisse
    cols_result = ["AGS", "Ort", "Partei", "Stimmart", "Stimmen", "Anteil"]
    df_results = pd.DataFrame(columns=cols_result)
    
    # DF Statistiken
    col_stats = ["AGS", "Ort", "WKNr", "Statistik", "Stimmen", "Anteil"]
    df_stats = pd.DataFrame(columns=col_stats)
    
    # Durchlauf durch alle <Gruppenergebnis>, sammeln der Gemeindedaten
    for elem in root: 
        ags = elem.attrib.get("GemNr")
        name = elem.attrib.get("GemName") 
        wk_nr = elem.attrib.get("KreisNr")
        # Unterpunkt ist <Gruppenergebnis>, hier findet sich unter "Position" die Beschreibung des jew. Datensatzes
        for sub in elem: 
            position = sub.attrib.get("Position")
            
            # Check ob position eine Nummer ist - dann handelt es sich nämlich um ein Partei-Stimmergebnis
            # dann wird das result-Dataframe befüllt
            if (has_numbers(position)) | (position == "UNG") | (position == "GUE"): 
                partei = parteien_dict[position]
                for node in sub: 
                    kind_of_votes = node.attrib.get("Stimmart")
                    if kind_of_votes == erststimmen: 
                        stimmart = "Erststimme"
                    else: 
                        stimmart = "Zweitstimme"
                    count_votes = node.attrib.get("Stimmen")
                    votes_share = node.attrib.get("Anteil")
                    ser_results = pd.Series([ags, name, partei, stimmart, count_votes, votes_share], index=cols_result)
                    df_results = df_results.append(ser_results, ignore_index=True)
            
            # wenn Buchstaben-Kennung wird das statistische Dataframe befüllt
            else:
                
                info_node = sub[0]
                stimmen = info_node.attrib.get("Stimmen")
                anteil = info_node.attrib.get("Anteil")
                
                if position == "WB": 
                    statistik = "Wahlberechtigte"
                    anteil = "100"
                elif position == "WAE":
                    statistik = "Wähler"
                else: 
                    print("Fehler")
               
                ser_stats = pd.Series([ags, name, wk_nr, statistik, stimmen, anteil], index=col_stats)
                df_stats = df_stats.append(ser_stats, ignore_index=True)
        
    return df_results, df_stats
                    
                    
            #if position
            
           

            
            
            
df_results_abfrage, df_daten_abfrage = read_xml(root)

In [7]:
df_results_abfrage

,AGS,Ort,Partei,Stimmart,Stimmen,Anteil
0,14730010,Arzberg,Ungültige Stimmen,Erststimme,7,0.9
1,14730010,Arzberg,Ungültige Stimmen,Zweitstimme,8,1.0
2,14730010,Arzberg,gültige Stimmen,Erststimme,793,99.1
3,14730010,Arzberg,gültige Stimmen,Zweitstimme,792,99.0
4,14730010,Arzberg,AfD,Erststimme,198,25.0
...,...,...,...,...,...,...
15414,14523460,Werda,Gesundheitsforschung,Zweitstimme,0,0.0
15415,14523460,Werda,Team Todenhöfer,Zweitstimme,0,0.0
15416,14523460,Werda,Volt,Zweitstimme,0,0.0
15417,14523460,Werda,ASMUSfddB,Erststimme,0,0.0


## richtige Datentypen vergeben

In [8]:
# richtige Datentypen vergeben
df_results_abfrage["Stimmen"] = df_results_abfrage["Stimmen"].astype(int)
df_results_abfrage["Anteil"] = df_results_abfrage["Anteil"].astype(float)

df_daten_abfrage["Stimmen"] = df_daten_abfrage["Stimmen"].astype(int)
df_daten_abfrage["Anteil"] = df_daten_abfrage["Anteil"].astype(float)

## Aus mehreren Stadt-Ergebnissen ein Ergebnis und einen Datensatz für jede Großstadt

In [9]:
# Stadtname und übergreifende AGS
city_dict = {"Leipzig":"14713000", "Dresden":"14612000"}

In [10]:
# Ergebnisliste

def group_cities_results(df, city_dict): 
    df_final = df.copy()
    
    for city in city_dict:
        df_ = df[df["Ort"].str.contains(city)]
        # Ort und AGS vereinheitlichen
        df_["Ort"] = city
        df_["AGS"] = city_dict[city]
        
        # über neuen, einheitlichen Ortsnamen gruppieren
        df_group = df_.groupby(["AGS", "Ort", "Partei", "Stimmart"], as_index=False).agg({"Stimmen":"sum"})
        
        
        # Anteile der Parteien neu ausrechnen
        # Erststimmen
        df_group.loc[df_group["Stimmart"]=="Erststimme", "Anteil"] = round(df_group.loc[df_group["Stimmart"]=="Erststimme"]["Stimmen"] / \
                                                    df_group.loc[(df_group["Stimmart"]=="Erststimme") & \
                                                             (df_group["Partei"]=="gültige Stimmen")]["Stimmen"]\
                                                                       .item() * 100,1)
        #Zweitstimmen
        df_group.loc[df_group["Stimmart"]=="Zweitstimme", "Anteil"] = round(df_group.loc[df_group["Stimmart"]=="Zweitstimme"]["Stimmen"] / \
                                                    df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & \
                                                             (df_group["Partei"]=="gültige Stimmen")]["Stimmen"]\
                                                                       .item() * 100,1)
        
        # gültige Erststimmen
        df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen"), "Anteil"] = round(\
            df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() /\
            (df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() + \
            df_group.loc[(df_group["Stimmart"]=="Erststimme") & (df_group["Partei"]=="Ungültige Stimmen")]["Stimmen"].item()) \
            * 100, 1) 
        # gültige Zweitstimmen
        df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen"), "Anteil"] = round(\
            df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() /\
            (df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="gültige Stimmen")]["Stimmen"].item() + \
            df_group.loc[(df_group["Stimmart"]=="Zweitstimme") & (df_group["Partei"]=="Ungültige Stimmen")]["Stimmen"].item()) \
            * 100, 1) 
        
        df_final = df_final[~df_final["Ort"].str.contains(city)]
        df_final = df_final.append(df_group)
    
    return df_final
    
df_ergebnisse = group_cities_results(df_results_abfrage, city_dict)    

<ipython-input-10-e684295ea4a7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Ort"] = city
<ipython-input-10-e684295ea4a7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["AGS"] = city_dict[city]


In [11]:
# Datenliste

def group_cities_data(df, city_dict): 
    # Kopie anfertigen, aus der später die einzelnen Datensätze entfernt und angehangen werden. 
    df_final = df.copy()
    
    for city in city_dict:
        df_ = df[df["Ort"].str.contains(city)]
        # Ort und AGS vereinheitlichen
        df_["Ort"] = city
        df_["AGS"] = city_dict[city]
        # Wahlkreis-Nummern herausfinden
        min_wk = df_["WKNr"].astype(int).min()
        max_wk =  df_["WKNr"].astype(int).max()
        df_["WKNr"] = f"{min_wk}-{max_wk}"
        
        # über neuen, einheitlichen Ortsnamen gruppieren
        df_group = df_.groupby(["AGS", "Ort", "WKNr", "Statistik"], as_index=False).agg({"Stimmen":"sum"})
        
        
        # Anteile der Parteien neu ausrechnen
        # Erststimmen
        df_group.loc[df_group["Statistik"]=="Wahlberechtigte", "Anteil"] = 100
        #Zweitstimmen
        df_group.loc[df_group["Statistik"]=="Wähler", "Anteil"] = round(df_group.loc[df_group["Statistik"]=="Wähler"]["Stimmen"] / \
                                                    df_group.loc[df_group["Statistik"]=="Wahlberechtigte"]["Stimmen"].item() * 100,1)
        
       
        df_final = df_final[~df_final["Ort"].str.contains(city)]
        df_final = df_final.append(df_group)
    
    return df_final

df_daten = group_cities_data(df_daten_abfrage, city_dict)

<ipython-input-11-367b36a6cd5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Ort"] = city
<ipython-input-11-367b36a6cd5c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["AGS"] = city_dict[city]
<ipython-input-11-367b36a6cd5c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

## Daten in Erst- und Zweitstimme teilen

In [15]:
# Daten Ergebnisse aufsplitten und zusätzlich eine "andere Parteien"-Version ausspielen

def split_up_data(df_ergebnisse): 
    df_ergebnisse_es = df_ergebnisse[df_ergebnisse["Stimmart"]=="Erststimme"]
    df_ergebnisse_zs = df_ergebnisse[df_ergebnisse["Stimmart"]=="Zweitstimme"]
    return df_ergebnisse_es, df_ergebnisse_zs

df_es, df_zs = split_up_data(df_ergebnisse)

# ZWEITSTIMME Daten von long auf wide umstellen

## "Andere Parteien" einrichten

In [16]:
def reduce_number_of_parties(df): 
    liste_standing_terms = ["Ungültige Stimmen", "gültige Stimmen", "CDU", "LINKE", \
                           "SPD", "AfD", "GRÜNE", "FDP"]


    sti = df.Stimmart.iloc[0]

    df_ = df[~df["Partei"].isin(liste_standing_terms)]
    df_rest = df[df["Partei"].isin(liste_standing_terms)]
    df_flat = df_.groupby(["AGS", "Ort"], as_index=False).agg({"Stimmen":"sum", "Anteil":"sum"})




    if sti == "Erststimme": 
        df_flat["Stimmart"] = "Erststimme"
        df_flat["Partei"] = "andere Parteien"
        df_flat = df_flat[['AGS', 'Ort', 'Partei', 'Stimmart', 'Stimmen', 'Anteil']]
        df_flat = df_flat.append(df_rest)
    elif sti == "Zweitstimme": 
        df_flat["Stimmart"] = "Zweitstimmen"
        df_flat["Partei"] = "andere Parteien"
        df_flat = df_flat[['AGS', 'Ort', 'Partei', 'Stimmart', 'Stimmen', 'Anteil']]
        df_flat = df_flat.append(df_rest)
    else: 
        print('Fehler')




    return df_flat

    
    
    
    
df_zs_flat = reduce_number_of_parties(df_zs)    

## Daten auf wide umstellen / Prefix vergeben

In [147]:
def results_long_to_wide(df_ergebnisse_es, df_ergebnisse_zs): 
    df_results_es_wide = pd.pivot(df_ergebnisse_es, index=["AGS", "Ort"], columns="Partei", values=["Stimmen", "Anteil"])\
        .reset_index().fillna("0")
    df_results_zs_wide = pd.pivot(df_ergebnisse_es, index=["AGS", "Ort"], columns="Partei", values=["Stimmen", "Anteil"])\
        .reset_index().fillna("0")
    # Multi-Index-Columns werden aufgelöst
    df_results_es_wide.columns = [" ".join(col).strip() for col in df_results_es_wide.columns.values]
    df_results_zs_wide.columns = [" ".join(col).strip() for col in df_results_zs_wide.columns.values]
    
    # Columns erhalten ein Präfix
    # Präfix _1 für Erststimme, _2 für Zweitstimme hinzufügen
    df_results_es_wide = df_results_es_wide.add_prefix("1_")
    df_results_zs_wide = df_results_zs_wide.add_prefix("2_")

    # Columns AgS und Ort Prefix entfernen
    df_results_es_wide.rename(columns={"1_AGS":"AGS", "1_Ort":"Ort"}, inplace=True)
    df_results_zs_wide.rename(columns={"2_AGS":"AGS", "2_Ort":"Ort"}, inplace=True)

    # komplette Result-Tabelle anlegen
    df_results_merge = pd.merge(df_results_es_wide, df_results_zs_wide, on=["AGS", "Ort"])
    
    df_results_merge.fillna(0)
    
    # convert Stimmen from float to int
    filter_stimmen = [col for col in df_results_merge if (col.startswith("1_Stimmen")) | (col.startswith("2_Stimmen")) ]
    df_results_merge[filter_stimmen] = df_results_merge[filter_stimmen].astype(int)
    
    return df_results_merge
 
df_results = results_long_to_wide(df_es_flat, df_zs_flat)   

## Korrekter Anteil für "andere Parteien" berechnen

In [148]:
def correct_anteil_andere_parteien(df_re):
    
    # erst Anteil für Erststimmen andere Parteinen
    df_re["1_Anteil andere Parteien"] = round(df_re["1_Stimmen andere Parteien"] / df_re["1_Stimmen gültige Stimmen"] * 100,1)
    df_re["2_Anteil andere Parteien"] = round(df_re["2_Stimmen andere Parteien"] / df_re["2_Stimmen gültige Stimmen"] * 100,1)
    
    # in float konvertieren
    df_re["1_Anteil andere Parteien"] = df_re["1_Anteil andere Parteien"].astype(float)
    df_re["2_Anteil andere Parteien"] = df_re["2_Anteil andere Parteien"].astype(float)
    
    
    
    return df_re

df_results_correct = correct_anteil_andere_parteien(df_results)


In [149]:
df_results_correct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419 entries, 0 to 418
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   AGS                          419 non-null    object 
 1   Ort                          419 non-null    object 
 2   1_Stimmen AfD                419 non-null    int32  
 3   1_Stimmen CDU                419 non-null    int32  
 4   1_Stimmen FDP                419 non-null    int32  
 5   1_Stimmen GRÜNE              419 non-null    int32  
 6   1_Stimmen LINKE              419 non-null    int32  
 7   1_Stimmen SPD                419 non-null    int32  
 8   1_Stimmen Ungültige Stimmen  419 non-null    int32  
 9   1_Stimmen andere Parteien    419 non-null    int32  
 10  1_Stimmen gültige Stimmen    419 non-null    int32  
 11  1_Anteil AfD                 419 non-null    float64
 12  1_Anteil CDU                 419 non-null    float64
 13  1_Anteil FDP        

## Statistik-DAten auf wide stellen, Ergebnisdaten mit Statistik-Daten verbinden

In [150]:
def statistic_long_to_wide(df): 
    
    df_wide = pd.pivot(df, index=["AGS", "Ort" ,"WKNr"], columns="Statistik", values=["Stimmen", "Anteil"])\
        .reset_index().fillna("0")
    # Multi-Index-Columns werden aufgelöst
    df_wide.columns = [" ".join(col).strip() for col in df_wide.columns.values]
    df_wide[["Stimmen Wahlberechtigte", "Stimmen Wähler"]] = \
        df_wide[["Stimmen Wahlberechtigte", "Stimmen Wähler"]].astype(int)
    
    return df_wide
    
df_daten_wide = statistic_long_to_wide(df_daten)

In [151]:
df_results[df_results["Ort"].str.contains("Dresden")]

,AGS,Ort,1_Stimmen AfD,1_Stimmen CDU,1_Stimmen FDP,1_Stimmen GRÜNE,1_Stimmen LINKE,1_Stimmen SPD,1_Stimmen Ungültige Stimmen,1_Stimmen andere Parteien,...,2_Stimmen gültige Stimmen,2_Anteil AfD,2_Anteil CDU,2_Anteil FDP,2_Anteil GRÜNE,2_Anteil LINKE,2_Anteil SPD,2_Anteil Ungültige Stimmen,2_Anteil andere Parteien,2_Anteil gültige Stimmen
183,14612000,Dresden,65747,32792,19156,20843,32974,65702,2691,26668,...,263882,24.9,12.4,7.3,7.9,12.5,24.9,1.0,10.1,99.0


In [152]:
df_daten_wide[df_daten_wide["Ort"].str.contains("Dresden")]

,AGS,Ort,WKNr,Stimmen Wahlberechtigte,Stimmen Wähler,Anteil Wahlberechtigte,Anteil Wähler
183,14612000,Dresden,159-160,426000,266573,100.0,62.6


## Ergebnisse und Statistik mergen

In [153]:
def merge_results_and_stats(df_results, df_stats): 
    # Results mit Statistik mergen
    df_all_merge = pd.merge(df_results, df_stats, on=["AGS", "Ort"])
    return df_all_merge

df_final = merge_results_and_stats(df_results, df_daten_wide)

In [154]:
df_final[df_final.Ort.str.contains("Dresden")]

,AGS,Ort,1_Stimmen AfD,1_Stimmen CDU,1_Stimmen FDP,1_Stimmen GRÜNE,1_Stimmen LINKE,1_Stimmen SPD,1_Stimmen Ungültige Stimmen,1_Stimmen andere Parteien,...,2_Anteil LINKE,2_Anteil SPD,2_Anteil Ungültige Stimmen,2_Anteil andere Parteien,2_Anteil gültige Stimmen,WKNr,Stimmen Wahlberechtigte,Stimmen Wähler,Anteil Wahlberechtigte,Anteil Wähler
183,14612000,Dresden,65747,32792,19156,20843,32974,65702,2691,26668,...,12.5,24.9,1.0,10.1,99.0,159-160,426000,266573,100.0,62.6


## Historisches Ergebnis mit einrechnen


In [155]:
dfh = wahlergebnisse_2021.copy()
dfh["AGS"]= dfh["AGS"].astype(str)
dfh = dfh.loc[:, dfh.columns != "Ort"]
dfh[dfh["AGS"].str.contains("14612000")]

,AGS,2017_AfD_%,2017_CDU_%,2017_LINKE_%,2017_SPD_%,2017_FDP_%,2017_GRÜNE_%,2017_andere_%
418,14612000,22.5,23.5,17.6,10.2,9.9,8.7,7.5


In [156]:
df_final = df_final.merge(dfh, on="AGS")

In [157]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419 entries, 0 to 418
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   AGS                          419 non-null    object 
 1   Ort                          419 non-null    object 
 2   1_Stimmen AfD                419 non-null    int32  
 3   1_Stimmen CDU                419 non-null    int32  
 4   1_Stimmen FDP                419 non-null    int32  
 5   1_Stimmen GRÜNE              419 non-null    int32  
 6   1_Stimmen LINKE              419 non-null    int32  
 7   1_Stimmen SPD                419 non-null    int32  
 8   1_Stimmen Ungültige Stimmen  419 non-null    int32  
 9   1_Stimmen andere Parteien    419 non-null    int32  
 10  1_Stimmen gültige Stimmen    419 non-null    int32  
 11  1_Anteil AfD                 419 non-null    float64
 12  1_Anteil CDU                 419 non-null    float64
 13  1_Anteil FDP        

In [158]:
df_final.columns

Index(['AGS', 'Ort', '1_Stimmen AfD', '1_Stimmen CDU', '1_Stimmen FDP',
       '1_Stimmen GRÜNE', '1_Stimmen LINKE', '1_Stimmen SPD',
       '1_Stimmen Ungültige Stimmen', '1_Stimmen andere Parteien',
       '1_Stimmen gültige Stimmen', '1_Anteil AfD', '1_Anteil CDU',
       '1_Anteil FDP', '1_Anteil GRÜNE', '1_Anteil LINKE', '1_Anteil SPD',
       '1_Anteil Ungültige Stimmen', '1_Anteil andere Parteien',
       '1_Anteil gültige Stimmen', '2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD',
       '2_Stimmen Ungültige Stimmen', '2_Stimmen andere Parteien',
       '2_Stimmen gültige Stimmen', '2_Anteil AfD', '2_Anteil CDU',
       '2_Anteil FDP', '2_Anteil GRÜNE', '2_Anteil LINKE', '2_Anteil SPD',
       '2_Anteil Ungültige Stimmen', '2_Anteil andere Parteien',
       '2_Anteil gültige Stimmen', 'WKNr', 'Stimmen Wahlberechtigte',
       'Stimmen Wähler', 'Anteil Wahlberechtigte', 'Anteil Wähler',
       '2017_AfD_%', '2017_CDU

## Gewinner bestimmen (wichtig für Einfärbung der Karte)

In [159]:
df_final["2_Gewinner"] = df_final[['2_Stimmen AfD', '2_Stimmen CDU',
       '2_Stimmen FDP', '2_Stimmen GRÜNE', '2_Stimmen LINKE', '2_Stimmen SPD']].idxmax(axis=1)
df_final["2_Gewinner"] = df_final["2_Gewinner"].str.split(" ")
df_final["2_Gewinner"] = df_final["2_Gewinner"].map(lambda x: x[1])



In [ ]:
# TODO: stimmen1 gewinner!!! 

In [160]:
df_final["2_Gewinner"]

0      SPD
1      SPD
2      AfD
3      SPD
4      AfD
      ... 
414    AfD
415    AfD
416    SPD
417    AfD
418    AfD
Name: 2_Gewinner, Length: 419, dtype: object

In [161]:
df_final.to_csv("wahlergebnisse_testdaten.csv", index=False)

In [110]:
df_final[df_final.Ort.str.contains("Schlema")]

,AGS,Ort,1_Stimmen AfD,1_Stimmen CDU,1_Stimmen FDP,1_Stimmen GRÜNE,1_Stimmen LINKE,1_Stimmen SPD,1_Stimmen Ungültige Stimmen,1_Stimmen andere Parteien,...,Anteil Wahlberechtigte,Anteil Wähler,2017_AfD_%,2017_CDU_%,2017_LINKE_%,2017_SPD_%,2017_FDP_%,2017_GRÜNE_%,2017_andere_%,2_Gewinner


# Parteien nach Stärke sortieren

Grundsätzlich haben wir **sechs freie Parteien-Slots** und andere Parteien. Die sechs gesetzten Parteien möchten wir jetzt nach Stärke verteilen.

**Zunächst nutzen wir dabei nur die Zweitstimmen**, bei Erststimmen müssen wir nochmal schauen 


Dafür nutze ich die Daten **vor** dem long to wide. Die Long-Daten finden sich in diesen Dataframes: 

df_es_flat 

df_zs_flat

## Zweitstimmen nach Stärke

In [45]:
def sort_parties(df): 
    
    cols = ["AGS", "Ort", "Gewinner",\
            "Partei_1_Stimmen", "Partei_1_Anteil", "Partei_1_Name",\
            "Partei_2_Stimmen", "Partei_2_Anteil", "Partei_2_Name",\
            "Partei_3_Stimmen", "Partei_3_Anteil", "Partei_3_Name",\
            "Partei_4_Stimmen", "Partei_4_Anteil", "Partei_4_Name",\
            "Partei_5_Stimmen", "Partei_5_Anteil", "Partei_5_Name",\
            "Partei_6_Stimmen", "Partei_6_Anteil", "Partei_6_Name",\
            "Partei_andere_Stimmen", "Partei_andere_Anteil", "Partei_andere_Name", \
           "ungültige Stimmen", "gültige Stimmen"]
            
           
      
    new_df = pd.DataFrame(columns=cols)
    
    # jeder Ort wird durchlaufen
    for ags in df["AGS"].unique():
        dict_var = {}
        df_ = df[df["AGS"]==ags]
        ort = df_.iloc[0].Ort
        
        
        # "andere Parteien" erfassen
        andere = df_[df_["Partei"]=="andere Parteien"]
        if andere.shape[0]>0:
            andere_stimmen = andere.Stimmen.item()
            andere_anteil = andere.Anteil.item()
            andere_name = "andere Parteien"
        else:
            andere_stimmen = 0
            andere_anteil = 0
            andere_name = "andere Parteien"
            
        
        # ungültige Stimmen erfassen
        ungültig = df_[df_["Partei"]=="Ungültige Stimmen"]["Stimmen"].item()
        gültig = df_[df_["Partei"]=="gültige Stimmen"]["Stimmen"].item()
        
        
        # andere Parteien herausfiltern (da sie sonst ev. einen Platz in der Parteienrangfolge einnehmen)
        df_ = df_[~df_.Partei.isin(["Ungültige Stimmen", "gültige Stimmen", "andere Parteien"])]
            
        # Parteien nach Stimmen sortieren, höchste zuerst
        df_sort = df_.sort_values(by="Stimmen", ascending=False)
        # Gewinner als Klarnamen herausfiltern
        gewinner = df_sort.iloc[0]["Partei"]
        
        # Durchlauf durch die AGS, für jede Gemeinde sechs Parteien in Rangfolge setzen
        for i in range(len(df_sort.index)):
            j = i+1
            
            j = i+1
            var_stimmen = f"partei_{j}_Stimmen"
            var_anteil = f"partei_{j}_Anteil"
            var_name = f"partei_{j}_Name"
            val_stimmen = df_sort.iloc[i]["Stimmen"]
            val_anteil = df_sort.iloc[i]["Anteil"]
            val_name = df_sort.iloc[i]["Partei"]
            
            dict_var[var_stimmen] = val_stimmen
            dict_var[var_anteil] = val_anteil
            dict_var[var_name] = val_name
            
        
        new_ser = pd.Series([ags, ort, gewinner, \
                             dict_var['partei_1_Stimmen'],dict_var['partei_1_Anteil'], dict_var['partei_1_Name'],\
                             dict_var['partei_2_Stimmen'],dict_var['partei_2_Anteil'], dict_var['partei_2_Name'],\
                             dict_var['partei_3_Stimmen'],dict_var['partei_3_Anteil'], dict_var['partei_3_Name'],\
                             dict_var['partei_4_Stimmen'],dict_var['partei_4_Anteil'], dict_var['partei_4_Name'],\
                             dict_var['partei_5_Stimmen'],dict_var['partei_5_Anteil'], dict_var['partei_5_Name'],\
                             dict_var['partei_6_Stimmen'],dict_var['partei_6_Anteil'], dict_var['partei_6_Name'],\
                             andere_stimmen, andere_anteil, andere_name, \
                             ungültig, gültig
                            ], index=cols)
        
        new_df = new_df.append(new_ser, ignore_index=True)
    
    # richtige Datentypen vergeben
    new_df[["Partei_andere_Stimmen", "ungültige Stimmen", "gültige Stimmen"]] = \
        new_df[["Partei_andere_Stimmen", "ungültige Stimmen", "gültige Stimmen"]].astype(int)
    
    new_df["Partei_andere_Anteil"] = round(new_df["Partei_andere_Stimmen"] / new_df["gültige Stimmen"] * 100,1)
    
    return new_df
            
            
        
        
        
    
    
df_zs_rank = sort_parties(df_zs_flat)

## Erststimme nach Stärke

In [46]:
df_es_rank = sort_parties(df_es_flat)   

In [47]:
# ACHTUNG, nur für Demozwecke
# df_es_rank["Gewinner"] = "Name, Vorname (" + df_es_rank['Gewinner'] + ")"
# df_es_rank["Partei_1_Name"] = "Name, Vorname (" + df_es_rank['Partei_1_Name'] + ")"
# df_es_rank["Partei_2_Name"] = "Name, Vorname (" + df_es_rank['Partei_2_Name'] + ")"
# df_es_rank["Partei_3_Name"] = "Name, Vorname (" + df_es_rank['Partei_3_Name'] + ")"
# df_es_rank["Partei_4_Name"] = "Name, Vorname (" + df_es_rank['Partei_4_Name'] + ")"
# df_es_rank["Partei_5_Name"] = "Name, Vorname (" + df_es_rank['Partei_5_Name'] + ")"
# df_es_rank["Partei_6_Name"] = "Name, Vorname (" + df_es_rank['Partei_6_Name'] + ")"

## Partei nach Stärke mit WK-Daten mergen

In [48]:
# Schritt 1 - präfix vergeben
#Präfix _1 für Erststimme, _2 für Zweitstimme hinzufügen
df_es_rank = df_es_rank.add_prefix("1_")
df_zs_rank = df_zs_rank.add_prefix("2_")
df_es_rank.rename(columns={"1_AGS":"AGS", "1_Ort":"Ort"}, inplace=True)
df_zs_rank.rename(columns={"2_AGS":"AGS", "2_Ort":"Ort"}, inplace=True)

In [49]:
# Schritt 2 - alle DAten zusammenfassen in eine Wide-Tabelle

df_merge = df_es_rank.merge(df_zs_rank, on=["AGS", "Ort"])
df_merge = df_merge.merge(df_daten_wide, on=["AGS", "Ort"])

In [50]:
df_merge.to_clipboard(decimal=",", index=False)
df_merge.to_csv("datawrapper_dummy.csv", index=False)

## Wahlergebnisse 2017 einarbeiten

In [53]:
df_merge["2_Partei_1_2017_Anteil"] = 0
df_merge["2_Partei_2_2017_Anteil"] = 0
df_merge["2_Partei_3_2017_Anteil"] = 0
df_merge["2_Partei_4_2017_Anteil"] = 0
df_merge["2_Partei_5_2017_Anteil"] = 0
df_merge["2_Partei_6_2017_Anteil"] = 0
df_merge["2_Partei_andere_2017_Anteil"] = 0
    

In [56]:
# dfh = wahlergebnisse_2021.copy()

# def create_old_results(df, dfh): 
#     for i in df.index: 
        
#         dict_col_names = {"AfD":"2017_AfD_%", "CDU":"2017_CDU_%", "LINKE":"2017_LINKE_%", "SPD":"2017_SPD_%", \
#                          "FDP":"2017_FDP_%", "GRÜNE":"2017_GRÜNE_%", "andere Parteien":"2017_andere_%"} 
#         partei_1 = df.at[i, "2_Partei_1_Name"]
#         anteil_2017_1 = 
        
        
        
# create_old_results(df_merge)

In [57]:
dfh

,AGS,Ort,2017_AfD_%,2017_CDU_%,2017_LINKE_%,2017_SPD_%,2017_FDP_%,2017_GRÜNE_%,2017_andere_%
0,14730010,Arzberg,31.2,28.4,14.5,10.4,7.9,1.1,6.5
1,14730020,"Bad Düben, Stadt",22.9,30.4,15.8,13.8,8.6,3.1,5.4
2,14730030,Beilrode,28.4,30.7,13.8,11.9,8.0,1.7,5.7
3,14730045,"Belgern-Schildau, Stadt",31.8,28.6,12.6,11.9,6.7,2.1,6.3
4,14730050,Cavertitz,32.4,31.0,13.0,8.4,7.3,1.9,6.1
...,...,...,...,...,...,...,...,...,...
419,14523420,Tirpersdorf,28.4,31.8,15.5,10.2,5.9,2.2,5.9
420,14523430,"Treuen, Stadt",26.4,31.5,15.3,10.7,6.8,3.1,6.2
421,14523440,Triebel/Vogtl.,26.3,30.5,16.7,11.3,6.0,2.5,6.8
422,14523450,Weischlitz,24.7,33.8,14.2,10.4,7.7,3.5,5.7


# Erstimmen-Daten mit Wahlkreis-Nr. und Kandidaten verbinden

In [447]:
df_k = pd.read_excel("./data/statistik-sachsen_bundestagswahl2021_direktbewerber.xlsx", header=1)

In [473]:
# nur Kandidaten der sechs größeren Parteien behalten
liste_parteien = ["AfD", "CDU", "DIE LINKE", "FDP", "GRÜNE", "SPD"]
df_kandidaten = df_k[df_k["Wahlvorschlag \nKurzbezeichnung"].isin(liste_parteien)] 
df_kandidaten["Full_Name"] = df_kandidaten['Name'] +" (" + df_kandidaten['Wahlvorschlag \nKurzbezeichnung']+")"

df_kandidaten.rename(columns={"Wahlkreis 151 Nordsachsen":"Wahlkreis", "Nummer":"Listennummer", \
                             "Wahlvorschlag \nKurzbezeichnung": "Partei", \
                              "Wahlvorschlag \nLangbezeichnung": "Partei_lang", \
                             "Name":"Name", "Full_Name": "Name_lang"}, inplace=True)
# richtige Formate vergeben
df_kandidaten["Wahlkreis"] = df_kandidaten["Wahlkreis"].astype(str)
df_kandidaten["Listennummer"] = df_kandidaten["Listennummer"].astype(str)

<ipython-input-473-9ed62e74cfbf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kandidaten["Full_Name"] = df_kandidaten['Name'] +" (" + df_kandidaten['Wahlvorschlag \nKurzbezeichnung']+")"
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-473-9ed62e74cfbf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [474]:
df_kandidaten

,Wahlkreis,Listennummer,Partei,Partei_lang,Name,Name_lang
0,151,1,AfD,Alternative für Deutschland,"Bochmann, René","Bochmann, René (AfD)"
1,151,2,CDU,Christlich Demokratische Union Deutschlands,"Schenderlein, Dr. Christiane","Schenderlein, Dr. Christiane (CDU)"
2,151,3,DIE LINKE,DIE LINKE,"Rubach, Philipp","Rubach, Philipp (DIE LINKE)"
3,151,4,SPD,Sozialdemokratische Partei Deutschlands,"Kleinke, Rüdiger","Kleinke, Rüdiger (SPD)"
4,151,5,FDP,Freie Demokratische Partei,"Richter, Martin Max","Richter, Martin Max (FDP)"
...,...,...,...,...,...,...
195,166,2,CDU,Christlich Demokratische Union Deutschlands,"Magwas, Yvonne","Magwas, Yvonne (CDU)"
196,166,3,DIE LINKE,DIE LINKE,"Höfer, Johannes","Höfer, Johannes (DIE LINKE)"
197,166,4,SPD,Sozialdemokratische Partei Deutschlands,"Burmeister, Kay","Burmeister, Kay (SPD)"
198,166,5,FDP,Freie Demokratische Partei,"Ludwig, André","Ludwig, André (FDP)"


In [487]:
def find_candidates(df_results, df_candidates):
    all_ags = df_results.AGS.unique()
    
    # Helper um mit Partie-Namen die Listennummer herauszufinden
    def return_party_nr(p_name): 
        for nr, name in parteien_dict.items():
            if p_name == name:
                return nr
        
    
    for ags in all_ags: 
        df_ = df_results[df_results["AGS"]==ags]
        winner = df_["1_Partei_1_Name"].item()
        number_winner = return_party_nr(winner)
        wknr = df_["WKNr"].item()
        name_winner = ""
        try:
            name_winner = df_kandidaten[(df_kandidaten["Wahlkreis"]==wknr) & \
                                    (df_kandidaten["Listennummer"]==number_winner)]["Name_lang"].item()
        except: 
            print(wknr)
            print(ags)
            print(number_winner)
            print(winner)
        percent_share_winner = 
    
        df_results.loc[df_results["AGS"]==ags, "1_Gewinner"] = name_winner + " (" ")"
    

    
    
    return df_results




        
df_fin = find_candidates(df_merge, df_kandidaten)

159-160
14612000
1
AfD
152-153
14713000
2
CDU


## Leipzig und Dresden haben je zwei Wahlkreise... Winner verändern

Hier muss ich auf Werte zurückgreifen, die 

In [472]:
wknr = "164"
number_winner = "2"
df_kandidaten[df_kandidaten["Listennummer"]==1]

,Wahlkreis,Listennummer,Partei,Partei_lang,Name,Name_lang
0,151,1,AfD,Alternative für Deutschland,"Bochmann, René","Bochmann, René (AfD)"
13,152,1,AfD,Alternative für Deutschland,"Neumann, Christoph","Neumann, Christoph (AfD)"
30,153,1,AfD,Alternative für Deutschland,"Droese, Siegbert","Droese, Siegbert (AfD)"
45,154,1,AfD,Alternative für Deutschland,"Naujok, Edgar","Naujok, Edgar (AfD)"
57,155,1,AfD,Alternative für Deutschland,"Lenk, Barbara","Lenk, Barbara (AfD)"
69,156,1,AfD,Alternative für Deutschland,"Hilse, Karsten","Hilse, Karsten (AfD)"
80,157,1,AfD,Alternative für Deutschland,"Chrupalla, Tino","Chrupalla, Tino (AfD)"
91,158,1,AfD,Alternative für Deutschland,"Janich, Steffen","Janich, Steffen (AfD)"
103,159,1,AfD,Alternative für Deutschland,"Maier, Jens","Maier, Jens (AfD)"
119,160,1,AfD,Alternative für Deutschland,"Harlaß, Andreas","Harlaß, Andreas (AfD)"
